# Estimating a Yield Curve Model

## Loading Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")
include("jlFiles/lagnPs.jl")

lagnPs (generic function with 2 methods)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

In [3]:
xx = readdlm("Data/USCMRatesPs.csv",',',header=true)
y  = xx[1]

YearMonth = 1970 + 1/24 + (1/12)*collect(0:size(y,1)-1)

y[:,1] = ((1-0.25*y[:,1]/100).^(-1/0.25) - 1)*100    #discount basis->effective
y[:,2] = ((1-0.50*y[:,2]/100).^(-1/0.5) - 1)*100

m      = [0.25 0.5 1 3 5 7 10]                             #time to maturity (in years)
mMonth = broadcast(x->round(Int,x),m*12)                   #maturites, in months (integers)
y      = log.(1+y/100)                    #continuously compounded interest rates
y      = y/12                            #interest rates per month (the period length of data)

(T,n) = size(y)

(520, 7)

## Establishing Notation for an Affine Yield Curve Model


An affine yield curve model implies that the yield on an $n$-period discount
bond is

$$
\begin{equation}
y_{nt}=a_{n}+b_{n}^{\prime}x_{t}\nonumber
\end{equation}
$$

where $x_{t}$ is an $K\times1$ vector of state variables. Here $a_{n}$ is a
scalar and $b_{n}$ is a $K\times1$ vector (so $b_{n}^{\prime}x_{t}$ is a scalar). 

Transpose to get
$$
y_{nt}=a_{n}+\underset{1\times K}{\underbrace{x_{t}^{\prime}}}
\underset{K\times1}{\underbrace{b_{n}}}
$$

As an example, consider two different maturities ($n$), denoted $1$ and $2$.
The row vector is then

$$
\begin{bmatrix}
y_{1t} & y_{2t}
\end{bmatrix} =
\begin{bmatrix}
a_{1} & a_{2}
\end{bmatrix}
+x_{t}^{\prime}
\begin{bmatrix}
b_{1} & b_{2}
\end{bmatrix}.
$$

Now, stack observations ($t=1$ to $T$) vertically

$$
Y_{T\times n}=\mathbf{1}_{T\times1}\otimes a_{1\times n}+X_{T\times K}b_{K\times n},
$$

where $\mathbf{1}_{T\times1}$ is a $T\times1$ vector of ones and $\otimes$ is
the Kronecker product.

An example of the Kronecker product:
$$
\mathbf{1}_{2\times1}\otimes
\begin{bmatrix}
1 & 2 & 3
\end{bmatrix} =
\begin{bmatrix}
1 & 2 & 3\\
1 & 2 & 3
\end{bmatrix}.
$$

### From Model Parameters to $a_{n}$ and $b_{n}$

The following function VasicekABFn.jl takes the
Vasicek model parameters as inputs and calculates the $a_{n}$ and $b_{n}$ values.

In [4]:
function VasicekABFn(lambda,mu,rho,sigma,nMo,nMu,yo)

  nMax = maximum([nMo;nMu])    #longest maturity to calculate (a,b) for
  Nvec = 1:nMax

  A = fill(NaN,(1,nMax))              #recursive solution of AR(1) model
  B = fill(NaN,(1,nMax))
  B[1] = 1 + 0*rho
  A[1] = 0 + 0*(1-rho)*mu - (lambda+0)^2*sigma^2/2
  for n = 2:nMax
    B[n] = 1 + B[n-1]*rho
    A[n] = A[n-1] + B[n-1]*(1-rho)*mu - (lambda+B[n-1])^2*sigma^2/2
  end

  a  = A./Nvec'
  b  = B./Nvec'

  ao = a[:,nMo][1]                       #required maturities
  bo = b[:,nMo][1]                       #[1] to make sure the are scalar

  xt    = (yo-ao)/bo                     #value of state variable x(t)

  au    = a[:,nMu]
  bu    = b[:,nMu]
  yuHat = au .+ xt*bu                   #Txn, fitted values of yu

  return ao,bo,xt,au,bu,yuHat

end

VasicekABFn (generic function with 1 method)

### Testing the VasicekABFn Function

In [6]:
vvo = 1                               #indices of yields that are observed without errors
vvu = 2:n                             #indices of yields with observation errors

yo = y[:,vvo]                         #observable yields
yu = y[:,vvu]                         #unobservable yields
nMo = mMonth[vvo]                     #maturity of yo
nMu = mMonth[vvu]                     #maturity of yu


#just testing the Vasicek functions
(ao,bo,xt,au,bu,yuHat) = VasicekABFn(1,0.5,0.9,0.02,nMo,nMu,yo)

println("ao and bo: ")
printmat([ao bo])

plot(YearMonth,xt,legend=false,xlims=(1970,2014))
title!("xt (implied by ao and bo, using specific parameter values)")

ao and bo: 
     0.047     0.903



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.050 
 
 
 -0.045 
 
 
 -0.040 
 
 
 xt (implied by ao and bo, using specific parameter values) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.8935,196.369 40.9404,212.722 41.9873,223.753 43.0342,226.399 44.0811,219.122 45.128,222.651 46.1748,227.721 47.2217,228.603 48.2686,234.991 49.3155,239.614 
 50.3624,253.469 51.4093,262.473 52.4562,271.907 53.503,288.118 54.5499,295.119 55.5968,284.615 56.6437,278.483 57.6906,265.107 58.7375,250.831 59.7844,260.937 
 60.8312,266.424 61.8781,271.468 62.925,276.73 63.9719,281.331 65.0188,295.119 66.0657,299.054 67.1126,287.461 68.1594,287.899 69.2063,288.337 70.2532,283.521 
 71.3001,281.988 72.347,281.112 73.3939,267.082 74.4408,265.326 75.4877,264.449 76.5345,258.082 77.5814,250.612 78.6283,246.434 79.6752,235.652 80.7221,231.908 
 81.769,229.705 82.8159,211.397 83.8627,193.272 84.9096,178.656 85.9565,187.074 87.0034,210.735 88.0503,197.254 89.0972,205.654 90.1441,198.581 91.1909,212.943 
 92.2378,194.378 93.2847,186.189 94.3316,188.403 95.3785,195.706 96.4254,203.444 97.4723,172.226 98.5191,192.166 99.566,205.433 100.613,205.212 101.66,212.28 
 102.707,231.908 103.754,248.633 104.8,248.853 105.847,246.214 106.894,254.567 107.941,252.15 108.988,234.771 110.035,227.942 111.082,228.382 112.129,238.514 
 113.176,249.073 114.222,249.952 115.269,262.473 116.316,262.254 117.363,259.619 118.41,262.693 119.457,255.226 120.504,250.612 121.551,254.567 122.598,256.545 
 123.644,257.862 124.691,261.376 125.738,265.107 126.785,273.88 127.832,267.959 128.879,266.862 129.926,268.398 130.973,269.714 132.02,260.497 133.066,259.18 
 134.113,255.446 135.16,248.853 136.207,241.815 137.254,234.11 138.301,235.432 139.348,236.092 140.395,227.942 141.441,227.721 142.488,231.247 143.535,231.247 
 144.582,228.603 145.629,221.548 146.676,215.371 147.723,213.826 148.77,196.812 149.817,193.715 150.863,179.321 151.91,169.564 152.957,163.571 154.004,164.238 
 155.051,160.685 156.098,161.129 157.145,157.797 158.192,170.008 159.239,166.013 160.285,159.796 161.332,143.344 162.379,111.238 163.426,109.227 164.473,103.64 
 165.52,104.534 166.567,85.288 167.614,32.7037 168.661,77.6673 169.707,180.651 170.754,214.046 171.801,192.166 172.848,168.455 173.895,143.121 174.942,113.025 
 175.989,65.7747 177.036,26.1646 178.082,36.76 179.129,41.9403 180.176,74.0788 181.223,66.6728 182.27,7.87402 183.317,43.2911 184.364,38.3369 185.411,25.7134 
 186.458,43.9665 187.504,70.0399 188.551,129.981 189.598,130.203 190.645,98.2726 191.692,71.3864 192.739,89.3198 193.786,88.8719 194.833,102.522 195.88,94.0212 
 196.926,119.052 197.973,178.435 199.02,195.263 200.067,199.908 201.114,191.944 202.161,194.821 203.208,196.59 204.255,191.059 205.302,185.746 206.348,188.846 
 207.395,189.288 208.442,175.996 209.489,169.564 210.536,163.794 211.583,171.339 212.63,179.321 213.677,176.661 214.724,171.339 215.77,173.557 216.817,169.342 
 217.864,159.796 218.911,156.019 219.958,152.908 221.005,152.018 222.052,146.459 223.099,138.669 224.145,140.895 225.192,154.908 226.239,179.986 227.286,192.166 
 228.333,198.802 229.38,187.517 230.427,181.98 231.474,194.6 232.521,204.991 233.567,216.695 234.614,213.826 235.661,212.501 236.708,213.384 237.755,212.059 
 238.802,210.293 239.849,213.384 240.896,214.046 241.943,214.267 242.989,225.297 244.036,236.312 245.083,234.331 246.13,233.009 247.177,241.375 248.224,247.974 
 249.271,255.007 250.318,255.666 251.365,251.93 252.411,247.974 253.458,250.172 254.505,246.654 255.552,246.654 256.599,245.555 257.646,245.115 258.693,244.895 
 259.74,244.455 260.787,236.753 261.833,228.823 262.88,234.771 263.927,244.455 264.974,242.695 266.021,241.815 267.068,245.115 268.115,244.235 269.162,239.614 
 270.208,231.908 271.255,227.501 272.302,221.548 273.349,214.267 274.396,210.293 275.443,207.

## The Likelihood Function

To formulate the log-liklihood, the VasicekTsCsFn
first takes the parameter vectors and transforms the parameters (scaling,
making sure that $-1\leq\rho\leq1$, etc) and the calls on the function
VasicekABFn (see above).


From the previous output, the code in VasicekTsCsFn moves on to
define the 1-step ahead forecast errors of $y_{ot}$ as

$$
v_{t}=y_{ot}-E_{t-1}y_{ot}
$$

and its variance $S=b_{o}^{\prime}\sigma^{2}b_{o}$. The contribution to the
likelihood function in $t$ is proportional to $v_{t}^{\prime}S^{-1}v_{t}$.
Then, the code defines the cross-sectional errors (for $y_{ut}$) as

$$
u_{t}=y_{ut}-\hat{y}_{ut} \ \text{, where } \ \hat{y}_{ut}=a_{u}+b_{u}^{\prime}x_{t},
$$

The contribution to the likelihood function is $u_{t}^{\prime}\Omega^{-1}
u_{t}$, where the code assumes that $\Omega$ is diagonal,
$
\Omega=I\omega^{2},
$
where $\omega$ is one of the parameters estimated. 

The function VasicekTsCsLossFn creates
the value of the function that will be minimized.

In [7]:
function VasicekTsCsFn(par,yo,yu,nMo,nMu)
#VasicekTsCsFn    Loss function for estimating the Vasicek model using both
#                 time series and cross sectional information

  J = length(nMu)
  T = size(yo,1)

  lambda  = par[1]*100
  mu      = par[2]/1200
  p       = 1 - 2/(1+exp(par[3]))          #inverse: par[3] = log((1+p)/(1-p))
  s2      = (par[4]/1200)^2
  omega_i = abs(par[5]/1200)

  if length(nMo) != 1
    error("yo must be a single yield")
  end

  (ao,bo,xt,au,bu,yuHat) = VasicekABFn(lambda,mu,p,sqrt(s2),nMo,nMu,yo)

  Et1xt    = (1-p)*mu + p*lagnPs(xt)    #E(t-1)x(t)
  Et1xt[1] = mu
  Et1yo    = ao + Et1xt*bo              #E(t-1)yo(t)
  v        = yo - Et1yo                 #Tx1, forecast error of yo
  S        = bo's2*bo                   #variance of forecast error of yo
  S_1      = inv(S*1000)*1000           #1000/1000 improves the precision a bit
  LLo_t    = -0.5*log(2*pi) - 0.5*log(det(S)) - 0.5*sum(v*S_1.*v,2)   #Tx1

  u        = yu -  yuHat                 #TxL, fitted errors of yu
  Omega    = eye(J)*omega_i^2
  Omega_1  = inv(Omega)                  #covariance matrix of u
  LLu_t    = -0.5*J*log(2*pi) - 0.5*log(det(Omega)) - 0.5*sum(u*Omega_1.*u,2)  #Tx1

  LL_t    = LLo_t + LLu_t                #Tx1, log likelihood(t), sum of TS and CS
  
  return LL_t,yuHat,u,xt

end


function VasicekTsCsLossFn(par,yo,yu,nMo,nMu)
  LL_t, = VasicekTsCsFn(par,yo,yu,nMo,nMu)
  MinusLL = -sum(LL_t[2:end])            #to be minimized 
  return MinusLL
end

VasicekTsCsLossFn (generic function with 1 method)

In [8]:
#notice: important that the elements of the parameter vector have similar magnitude
#lambda/100,mu*1200,p = 1 - 2/(1+exp(par(3))) so 5.6 -> p = 0.993,sigma*1200,omega*1200
par0 = [-2.3;10;5.5;0.5;0.8]

MinusLL = VasicekTsCsLossFn(par0,yo,yu,nMo,nMu)

printlnPs("The negative of the log-likelihood function at the starting guess: ",MinusLL)

The negative of the log-likelihood function at the starting guess: -21632.802


In [9]:
                             #do MLE by optimization with optimize, minimize -sum(LL)
Sol = optimize(par->VasicekTsCsLossFn(par,yo,yu,nMo,nMu),par0)
par1 = Optim.minimizer(Sol)

println("\npar0 and par1")
printmat([par0 par1])

#converting back to model parameters 
(lambda,mu,rho,s2) = (par1[1]*100,par1[2]/1200,1 - 2/(1+exp(par1[3])),par1[4]/1200)
println("Estimated (lambda,mu,rho,s2):")
printmat([lambda mu rho s2])


par0 and par1
    -2.300    -2.441
    10.000     9.672
     5.500     5.753
     0.500     0.525
     0.800     0.797

Estimated (lambda,mu,rho,s2):
  -244.095     0.008     0.994     0.000



In [10]:
                                               #fitted yields at parameter estimates
MinusLL = VasicekTsCsLossFn(par1,yo,yu,nMo,nMu)
println("The negative of the log-likelihood function at the point estimate: ",MinusLL)

(LL_t,yuHat,u,xt) = VasicekTsCsFn(par1,yo,yu,nMo,nMu)

yhatTsCs = [yo yuHat]                      #also yo

                                            #a and b at estimated parameters, average yield curve
(ao,bo,xt,au,bu,yuHatAvg) = VasicekABFn(lambda,mu,rho,s2,nMo,nMu,mean(yo))
yuHatAvg = [mean(yo) yuHatAvg]             #also yo



yy = y[:,[3,6]] - yhatTsCs[:,[3,6]]
  
plot(YearMonth,12*yy[:,1],label="1 year")
plot!(YearMonth,12*yy[:,2],line=(:dot,2),label="7 years")
plot!(xlims= (1970,2014),ylims=(-0.04,0.04))
title!("Pricing errors, Vasicek (TS and CS)")

The negative of the log-likelihood function at the point estimate: -21671.211618944784


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.02 
 
 
 0.00 
 
 
 0.02 
 
 
 Pricing errors, Vasicek (TS and CS) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.8935,199.906 40.9404,188.177 41.9873,191.936 43.0342,182.799 44.0811,168.545 45.128,169.788 46.1748,178.443 47.2217,181.715 48.2686,179.352 49.3155,182.698 
 50.3624,193.836 51.4093,197.481 52.4562,196.915 53.503,193.565 54.5499,188.104 55.5968,182.888 56.6437,163.411 57.6906,164.765 58.7375,176.578 59.7844,166.371 
 60.8312,171.93 61.8781,183.168 62.925,182.861 63.9719,176.579 65.0188,162.527 66.0657,155.006 67.1126,161.173 68.1594,147.816 69.2063,160.696 70.2532,157.953 
 71.3001,159.763 72.347,160.674 73.3939,165.898 74.4408,169.446 75.4877,181.912 76.5345,184.086 77.5814,183.403 78.6283,179.092 79.6752,172.951 80.7221,180.517 
 81.769,183.282 82.8159,202.601 83.8627,194.088 84.9096,205.777 85.9565,209.941 87.0034,200.161 88.0503,220.274 89.0972,215.888 90.1441,223.883 91.1909,217.578 
 92.2378,218.13 93.2847,198.847 94.3316,187.737 95.3785,177.54 96.4254,156.511 97.4723,196.457 98.5191,176.405 99.566,183.674 100.613,200.421 101.66,200.817 
 102.707,181.361 103.754,183.572 104.8,177.601 105.847,149.496 106.894,154.174 107.941,163.298 108.988,163.775 110.035,152.813 111.082,149.83 112.129,162.95 
 113.176,161.042 114.222,154.611 115.269,162.839 116.316,159.025 117.363,151.598 118.41,157.712 119.457,152.418 120.504,156.663 121.551,162.231 122.598,166.733 
 123.644,170.88 124.691,178.284 125.738,179.725 126.785,179.154 127.832,173.961 128.879,168.478 129.926,164.093 130.973,163.998 132.02,165.554 133.066,169.921 
 134.113,171.505 135.16,166.569 136.207,174.017 137.254,171.006 138.301,169.179 139.348,167.423 140.395,170.427 141.441,168.352 142.488,162.488 143.535,156.61 
 144.582,146.457 145.629,149.44 146.676,149.43 147.723,155.88 148.77,176.55 149.817,162.109 150.863,155.405 151.91,163.255 152.957,170.872 154.004,176.472 
 155.051,183.243 156.098,187.664 157.145,194.397 158.192,192.292 159.239,197.485 160.285,196.092 161.332,194.228 162.379,194.503 163.426,200.571 164.473,228.337 
 165.52,223.31 166.567,187.996 167.614,219.698 168.661,227.997 169.707,178.196 170.754,161.682 171.801,185.524 172.848,167.95 173.895,167.106 174.942,188.888 
 175.989,218.338 177.036,269.648 178.082,279.738 179.129,249.948 180.176,218.965 181.223,209.814 182.27,255.11 183.317,235.822 184.364,212.208 185.411,198.93 
 186.458,169.774 187.504,161.404 188.551,157.829 189.598,139.768 190.645,145.955 191.692,184.15 192.739,178.661 193.786,178.379 194.833,176.178 195.88,164.415 
 196.926,146.751 197.973,99.3816 199.02,88.8978 200.067,142.165 201.114,164.86 202.161,169.383 203.208,177.826 204.255,176.571 205.302,182.345 206.348,178.562 
 207.395,180.978 208.442,176.489 209.489,167.344 210.536,165.526 211.583,165.393 212.63,163.608 213.677,163.65 214.724,167.44 215.77,171.563 216.817,174.342 
 217.864,171.158 218.911,166.172 219.958,141.673 221.005,126.546 222.052,139.793 223.099,163.993 224.145,169.18 225.192,168.418 226.239,161.854 227.286,157.398 
 228.333,156.787 229.38,168.442 230.427,156.182 231.474,160.321 232.521,167.494 233.567,171.358 234.614,174.648 235.661,169.391 236.708,166.772 237.755,171.953 
 238.802,180.952 239.849,183.488 240.896,179.639 241.943,184.217 242.989,186.29 244.036,188.947 245.083,184.067 246.13,183.357 247.177,185.922 248.224,187.032 
 249.271,179.633 250.318,180.432 251.365,184.571 252.411,189.586 253.458,188.977 254.505,188.422 255.552,185.445 256.599,167.729 257.646,147.504 258.693,156.382 
 259.74,162.339 260.787,163.139 261.833,152.288 262.88,143.619 263.927,150.523 264.974,145.237 266.021,154.594 267.068,162.696 268.115,161.516 269.162,158.198 
 270.208,157.373 271.255,162.503 272.302,163.642 273.349,160.819 274.396,172.185 275.443,176.261 276.49,179.171 277.537,171.886

In [11]:
plot(m',12*yuHatAvg',label="at average x[t] ")
plot!(m',12*mean(y,1)',line=(:dot,2),label="data")

plot!(xlims=(0,11),ylims=(0.05,0.08))
title!("Avg yield curve, Vasicek (TS and CS)")
xlabel!("Maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.050 
 
 
 0.055 
 
 
 0.060 
 
 
 0.065 
 
 
 0.070 
 
 
 0.075 
 
 
 Avg yield curve, Vasicek (TS and CS) 
 
 
 Maturity (years) 
 
 
 
 
 
 
 
 at average x[t] 
 
 
 
 data

In [12]:
bOls = [ones(T) y[:,1]]\y                 #LS of yields on short yield
alfa = bOls[1:1,:]
beta = bOls[2:2,:]
                         #rescaling Vasicek au and bu to be comaparable with OLS on yo
bu_b = bu/bo             #y = au + bu*xt, but xt = (yo-ao)/bo
au_b = au - bu/bo*ao     #y = au + bu*(yo-ao)/bo = y = au - bu*ao/bo + bu/bo*yo
bu_b = [1 bu_b]          #also yo
au_b = [0 au_b]


plot(m',alfa'*1200,label="Vasicek")
plot!(m',au_b'*1200,line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Intercept times 1200, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Intercept times 1200, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS

In [13]:
plot(m',beta',label="Vasicek")
plot!(m',bu_b',line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Slope on short rate, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 Slope on short rate, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS